In [1]:
# file to handle recurrent neural network part of PHLUID model
# notice similarity to TP RNN

import os
import numpy as np
from keras.models import Model, Sequential, load_model
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import wandb
from wandb.keras import *
# paths
MAIN_DIR = '/media/oscar47/Oscar_Extra/Physics_data/swarm_data'
DATA_DIR= os.path.join(MAIN_DIR, 'rnn_output')
MAX_FRAME = 20 # number of consecutive frame fv groupings


# load datasets
x_train = np.load(os.path.join(DATA_DIR, 'train_x.npy'))
y_train = np.load(os.path.join(DATA_DIR, 'train_y.npy'))
x_val = np.load(os.path.join(DATA_DIR, 'val_x.npy'))
y_val = np.load(os.path.join(DATA_DIR, 'val_y.npy'))

# build model functions--------------------------------
def train_custom(LSTM_layer_size_1=128,  LSTM_layer_size_2=128, LSTM_layer_size_3=128, 
              LSTM_layer_size_4=128, LSTM_layer_size_5=128, 
              dropout=0.1, learning_rate=0.01, epochs=1, batchsize=32):
    #initialize the neural net; 
    global model
    model = build_model(LSTM_layer_size_1,  LSTM_layer_size_2, LSTM_layer_size_3, 
            LSTM_layer_size_4, LSTM_layer_size_5, 
            dropout, learning_rate)
    
    #now run training
    history = model.fit(
    x_train, y_train,
    batch_size = batchsize,
    validation_data=(x_val, y_val),
    epochs=epochs,
    callbacks=callbacks #use callbacks to have w&b log stats; will automatically save best model                     
    )

# define model
def build_model(LSTM_layer_size_1,  LSTM_layer_size_2, LSTM_layer_size_3, 
          LSTM_layer_size_4, LSTM_layer_size_5, 
          dropout, learning_rate):
    # call initialize function
    
    model = Sequential()
    # RNN layers for language processing
    model.add(LSTM(LSTM_layer_size_1, input_shape = (x_train[0].shape), return_sequences=True))
    model.add(LSTM(LSTM_layer_size_2, return_sequences=True))
    model.add(LSTM(LSTM_layer_size_3, return_sequences=True))
    model.add(LSTM(LSTM_layer_size_4, return_sequences=True))
    model.add(LSTM(LSTM_layer_size_5))
    model.add(Dropout(dropout))

    model.add(Dense(len(y_train[0])))
    model.add(Activation('softmax'))


    # put structure together
    optimizer = RMSprop(learning_rate = learning_rate)
    model.compile(loss='categorical_crossentropy')

    return model

def train(config=None):
    with wandb.init(config=config):
    # If called by wandb.agent, as below,
    # this config will be set by Sweep Controller
        config = wandb.config

        #pprint.pprint(config)

        #initialize the neural net; 
        global model
        model = build_model(config.LSTM_layer_size_1,  config.LSTM_layer_size_2, config.LSTM_layer_size_3, 
              config.LSTM_layer_size_4, config.LSTM_layer_size_5, 
              config.dropout, config.learning_rate)
      
      
    #now run training
    history = model.fit(
    x_train, y_train,
    batch_size = config.batch_size,
    validation_data=(x_val, y_val),
    shuffle=False,
    epochs=config.epochs,
    callbacks=[WandbCallback()] #use callbacks to have w&b log stats; will automatically save best model                     
    )

def train_custom_resume(model, batchsize, epochs):
   #now run training
   history = model.fit(
   x_train, y_train,
   batch_size = batchsize,
   validation_data=(x_val, y_val),
   epochs=epochs,
   callbacks=callbacks #use callbacks to have w&b log stats; will automatically save best model                     
   )

# define two other callbacks
# save model
# if no directory "models" exists, create it
if not(os.path.exists(os.path.join(MAIN_DIR, 'models'))):
    os.mkdir(os.path.join(MAIN_DIR, 'models'))
modelpath = os.path.join(MAIN_DIR, 'models', 'rnn1.hdf5')
checkpoint = ModelCheckpoint(modelpath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')
# if learning stals, reduce the LR
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)

# compile the callbacks
#callbacks = [checkpoint, reduce_lr, WandbCallback()]
#callbacks = [checkpoint, reduce_lr]

# set dictionary with random search; optimizing val_loss--------------------------
sweep_config= {
    'method': 'random',
    'name': 'val_accuracy',
    'goal': 'maximize'
}

sweep_config['metric']= 'val_accuracy'

parameters_dict = {
    'epochs': {
        'distribution': 'int_uniform',  #we want to specify a distribution type to more efficiently iterate through these hyperparams
       'min': 10,
       'max': 15
    },
    # for build_dataset
     'batch_size': {
       'distribution': 'int_uniform',  #we want to specify a distribution type to more efficiently iterate through these hyperparams
       'min': 64,
       'max': 128
    },
    'LSTM_layer_size_1': {
       'distribution': 'int_uniform',
       'min': 64,
       'max': 256
    },
    'LSTM_layer_size_2': {
       'distribution': 'int_uniform',
       'min': 64,
       'max': 256
    },
    'LSTM_layer_size_3': {
       'distribution': 'int_uniform',
       'min': 64,
       'max': 256
    },
    'LSTM_layer_size_4': {
       'distribution': 'int_uniform',
       'min': 64,
       'max': 256
    },
    'LSTM_layer_size_5': {
       'distribution': 'int_uniform',
       'min': 64,
       'max': 256
    },
     'dropout': {
             'distribution': 'uniform',
       'min': 0,
       'max': 0.6
    },
    'learning_rate':{
         #uniform distribution between 0 and 1
         'distribution': 'uniform', 
         'min': 0,
         'max': 0.1
     }
}

# append parameters to sweep config
sweep_config['parameters'] = parameters_dict 


# login to wandb----------------
wandb.init(project="PHLUID-SwarmRNN1", entity="oscarscholin")

# initialize sweep agent
sweep_id = wandb.sweep(sweep_config, project='PHLUID-SwarmRNN1', entity="oscarscholin")
wandb.agent(sweep_id, train, count=100)


2023-01-06 01:49:36.767393: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-06 01:49:36.835160: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-06 01:49:37.076073: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-06 01:49:37.076102: W tensorflow/compiler/xla/stream_executor/platform/default/dso_lo

Error: You must call wandb.init() before WandbCallback()